In [35]:
#include autoreload modules for fast testing and update
%load_ext autoreload
%autoreload 2

#import relevant modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sbn
import time

#import modules
from src.data.helper_closed_transactions import read_epex_file, filter_lead_time, extract_transactions
from src.visualization.visualize_transactions import executed_transactions_bar_chart, executed_transactions_heatmap_summary, executed_transactions_time_series

from src.data.welfare_complete import read_NTC_file, NTC_preparation
#set maximum columns display pandas
pd.options.display.max_columns = 50

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Bug Identification

Validate the transaction derived with given transactions for April 1 2019

In [2]:
df_updated_transactions=pd.read_csv("../data/processed/EPEX_spot_continous_complete_pipeline_2019_17-04-2021 18:38:10/2019-04/Updated Transactions/DE_20190401.csv",)

In [3]:
df_transactions=pd.read_csv("../data/interim/EPEX_spot_continous_complete_pipeline_2019_17-04-2021 18:38:10/2019-04/Transactions/DE_20190401.csv")

In [4]:
df_updated_transactions.shape

(25414, 31)

In [5]:
df_transactions.shape

(25414, 27)

In [6]:
df_transactions.head()

,End Validity Date,Executed Price & Volume,Delivery Start,Executed Volume_B,Execution Price_B,Initial ID_B,Instrument Type_B,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,index_B,lead_time_B,Executed Volume_S,Execution Price_S,Initial ID_S,Instrument Type_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,index_S,lead_time_S
0,2019-03-31 23:00:01.357000+00:00,"(37.23, 0.5)",2019-04-01 00:00:00+00:00,0.50,37.23,10391281385,Quarter Hour,2,N,10391281401,1.039128e+10,39.80,0.65,51413,0 days 00:59:58.643000,0.50,37.23,10391281397,Quarter Hour,1,N,10391281397,NaN,37.23,0.50,51409,0 days 00:59:58.719000
1,2019-03-31 23:00:02.583000+00:00,"(46.73, 0.15)",2019-04-01 00:00:00+00:00,0.15,46.73,10391281385,Quarter Hour,1,N,10391281419,1.039128e+10,46.73,0.15,51422,0 days 00:59:57.417000,0.15,46.73,10391281396,Quarter Hour,2,N,10391281396,NaN,46.73,0.50,51408,0 days 00:59:58.719000
2,2019-03-31 23:00:46.106000+00:00,"(43.65, 0.15)",2019-04-01 00:00:00+00:00,0.15,43.65,10391281545,Quarter Hour,1,N,10391281545,NaN,43.65,0.15,51509,0 days 00:59:46.616000,0.15,43.65,10391281772,Quarter Hour,1,N,10391281772,NaN,43.65,0.15,51649,0 days 00:59:13.894000
3,2019-03-31 23:01:40.633000+00:00,"(26.3, 1.0)",2019-04-01 00:00:00+00:00,1.00,26.30,10391281822,Hour,2,N,10391281822,NaN,26.30,3.00,51688,0 days 00:59:02.996000,1.00,26.30,10391281997,Hour,1,N,10391281997,NaN,25.30,1.00,51811,0 days 00:58:19.367000
4,2019-03-31 23:01:40.733000+00:00,"(26.3, 1.0)",2019-04-01 00:00:00+00:00,1.00,26.30,10391281822,Hour,2,N,10391281822,NaN,26.30,2.00,51689,0 days 00:58:19.367000,1.00,26.30,10391281999,Hour,1,N,10391281999,NaN,25.30,1.00,51812,0 days 00:58:19.267000


In [7]:
df_transactions[df_transactions['Instrument Type_B']=='Hour']['Executed Volume_S'].sum()

45575.600000000006

In [8]:
df_filtered_transactions=pd.read_csv("../data/interim/EPEX_spot_continous_complete_pipeline_2019_17-04-2021 18:38:10/2019-04/Filtered Orders/DE_20190401.csv")

In [9]:
df_filtered_transactions.head()

,index,Order ID,Initial ID,Parent ID,Start Validity Date,End Validity Date,Cancelling Date,Delivery Date,Instrument Type,Delivery Instrument,Is block,Area,Side,Price,Volume,Is Executed,Execution Price,Executed Volume,Executed Price & Volume,lead_time,Delivery Start
0,51408,10391281396,10391281396,NaN,2019-03-31 23:00:01.281000+00:00,2019-03-31 23:00:02.583000+00:00,NaN,2019-04-01 00:00:00+00:00,Quarter Hour,0 days 00:00:00,N,DE-TPS,S,46.73,0.50,2,46.73,0.150,"(46.73, 0.15)",0 days 00:59:58.719000,2019-04-01 00:00:00+00:00
1,51409,10391281397,10391281397,NaN,2019-03-31 23:00:01.281000+00:00,2019-03-31 23:00:01.357000+00:00,NaN,2019-04-01 00:00:00+00:00,Quarter Hour,0 days 00:00:00,N,DE-TPS,S,37.23,0.50,1,37.23,0.500,"(37.23, 0.5)",0 days 00:59:58.719000,2019-04-01 00:00:00+00:00
2,51410,10391281398,10391281398,NaN,2019-03-31 23:00:01.281000+00:00,2019-03-31 23:08:14.735000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-TPS,B,26.00,10.00,1,26.00,10.000,"(26.0, 10.0)",0 days 00:59:58.719000,2019-04-01 00:00:00+00:00
3,51413,10391281401,10391281385,1.039128e+10,2019-03-31 23:00:01.357000+00:00,2019-03-31 23:00:01.357000+00:00,NaN,2019-04-01 00:00:00+00:00,Quarter Hour,0 days 00:00:00,N,DE-AMP,B,39.80,0.65,2,37.23,0.500,"(37.23, 0.5)",0 days 00:59:58.643000,2019-04-01 00:00:00+00:00
4,51417,10391281406,10391281406,NaN,2019-03-31 23:00:01.702000+00:00,2019-03-31 23:03:28.937000+00:00,NaN,2019-04-01 00:00:00+00:00,Quarter Hour,0 days 00:00:00,N,DE-AMP,B,28.20,1.25,2,28.20,0.725,"(28.2, 0.725)",0 days 00:59:58.298000,2019-04-01 00:00:00+00:00


In [10]:
df_filtered_transactions['Delivery Start'].iloc[0]

'2019-04-01 00:00:00+00:00'

In [11]:
df_filtered_transactions[(df_filtered_transactions['Delivery Start']=='2019-04-01 00:00:00+00:00')&
                        (df_filtered_transactions['Instrument Type']=='Hour')]

,index,Order ID,Initial ID,Parent ID,Start Validity Date,End Validity Date,Cancelling Date,Delivery Date,Instrument Type,Delivery Instrument,Is block,Area,Side,Price,Volume,Is Executed,Execution Price,Executed Volume,Executed Price & Volume,lead_time,Delivery Start
2,51410,10391281398,10391281398,NaN,2019-03-31 23:00:01.281000+00:00,2019-03-31 23:08:14.735000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-TPS,B,26.00,10.0,1,26.00,10.0,"(26.0, 10.0)",0 days 00:59:58.719000,2019-04-01 00:00:00+00:00
8,51590,10391281708,10391281708,NaN,2019-03-31 23:18:45.651000+00:00,2019-03-31 23:18:46.792000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-AMP,B,21.10,8.0,2,21.10,8.0,"(21.1, 8.0)",0 days 00:41:14.349000,2019-04-01 00:00:00+00:00
9,51592,10391281708,10391281708,NaN,2019-03-31 23:18:46.792000+00:00,2019-03-31 23:18:47.628000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-AMP,B,21.10,10.0,2,21.10,10.0,"(21.1, 10.0)",0 days 00:41:13.208000,2019-04-01 00:00:00+00:00
10,51594,10391281708,10391281708,NaN,2019-03-31 23:18:47.628000+00:00,2019-03-31 23:18:48.273000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-AMP,B,21.10,1.0,1,21.10,1.0,"(21.1, 1.0)",0 days 00:41:12.372000,2019-04-01 00:00:00+00:00
11,51595,10391281708,10391281708,NaN,2019-03-31 23:00:34.345000+00:00,2019-03-31 23:18:39.772000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-AMP,B,21.10,10.0,2,21.10,1.1,"(21.1, 1.1)",0 days 00:59:25.655000,2019-04-01 00:00:00+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
663,57613,10391288390,10391288390,NaN,2019-03-31 23:29:08.558000+00:00,2019-03-31 23:29:08.558000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-AMP,S,23.00,5.0,1,23.10,5.0,"(23.1, 5.0)",0 days 00:30:51.442000,2019-04-01 00:00:00+00:00
686,57812,10391288549,10391288175,1.039129e+10,2019-03-31 23:29:50.338000+00:00,2019-03-31 23:29:50.338000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-TPS,B,23.99,0.6,1,23.99,0.6,"(23.99, 0.6)",0 days 00:30:09.662000,2019-04-01 00:00:00+00:00
687,57814,10391288550,10391288205,1.039129e+10,2019-03-31 23:29:50.421000+00:00,2019-03-31 23:29:59.525000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-TPS,S,23.99,4.8,2,23.99,4.0,"(23.99, 4.0)",0 days 00:30:09.579000,2019-04-01 00:00:00+00:00
690,57843,10391288578,10391288578,NaN,2019-03-31 23:29:59.525000+00:00,2019-03-31 23:29:59.525000+00:00,NaN,2019-04-01 00:00:00+00:00,Hour,0 days 00:00:00,N,DE-50Hz,B,28.80,4.0,1,23.99,4.0,"(23.99, 4.0)",0 days 00:30:00.475000,2019-04-01 00:00:00+00:00


The error is in the definition of the lead time: ```df['lead_time'] = df['Delivery Start']-df['Start Validity Date']```. There has been a typo: ```Start Validity Date``` should be replaced with ```End Validity Date```

The filtering function was correct but the multi-index pivoting was problematic.

Run updated pipeline for the first of April

In [2]:
# loading the daily csv file at day
df_new = read_epex_file('../data/external/EPEX_spot_continous_2019/DE Continuous Orders 2019-04/DE Continuous Orders 20190401.csv')

# filtering the daily csv file for the window of time of interest
df_filtered_new = filter_lead_time(df_new)

# derive transactions
pivoted_new, pivoted_levels_new = extract_transactions(df_filtered_new)

In [3]:
pivoted_new.head(30)

Executed Volume  \
End Validity Date                Executed Price & Volume Delivery Start            Side                   
2019-03-31 23:00:01.357000+00:00 (37.23, 0.5)            2019-04-01 00:00:00+00:00 B                0.5   
                                                                                   S                0.5   
2019-03-31 23:00:02.583000+00:00 (46.73, 0.15)           2019-04-01 00:00:00+00:00 B               0.15   
                                                                                   S               0.15   
2019-03-31 23:00:46.106000+00:00 (43.65, 0.15)           2019-04-01 00:00:00+00:00 B               0.15   
                                                                                   S               0.15   
2019-03-31 23:01:40.633000+00:00 (26.3, 1.0)             2019-04-01 00:00:00+00:00 B                  1   
                                                                                   S                  1   
2019-03-31 23:01:40.733000+00:00 (26.3, 1.0)             2019-04-01 00:00:00+00:00 B                  1   
                                                                                   S                  1   
2019-03-31 23:02:16.216000+00:00 (26.3, 1.0)             2019-04-01 00:00:00+00:00 B                  1   
                                                                                   S                  1   
2019-03-31 23:02:58.626000+00:00 (26.7, 1.0)             2019-04-01 00:00:00+00:00 B                  1   
                                                                                   S                  1   
2019-03-31 23:03:28.845000+00:00 (26.7, 4.9)             2019-04-01 00:00:00+00:00 B                4.9   
                                                                                   S                4.9   
2019-03-31 23:03:28.937000+00:00 (26.7, 4.9)             2019-04-01 00:00:00+00:00 B                4.9   
                                                                                   S                4.9   
                                 (28.2, 0.725)           2019-04-01 00:00:00+00:00 B              0.725   
                                                                                   S              0.725   
                                 (30.48, 0.5)            2019-04-01 00:00:00+00:00 B                0.5   
                                                                                   S                0.5   
2019-03-31 23:03:28.943000+00:00 (26.7, 3.0)             2019-04-01 00:00:00+00:00 B                  3   
                                                                                   S                  3   
                                 (28.2, 0.525)           2019-04-01 00:00:00+00:00 B              0.525   
                                                                                   S              0.525   
2019-03-31 23:03:28.944000+00:00 (26.7, 2.0)             2019-04-01 00:00:00+00:00 B                  2   
                                                                                   S                  2   
2019-03-31 23:03:42.368000+00:00 (26.5, 1.0)             2019-04-01 00:00:00+00:00 B                  1   
                                                                                   S                  1   

                                                                                        Execution Price  \
End Validity Date                Executed Price & Volume Delivery Start            Side                   
2019-03-31 23:00:01.357000+00:00 (37.23, 0.5)            2019-04-01 00:00:00+00:00 B              37.23   
                                                                                   S              37.23   
2019-03-31 23:00:02.583000+00:00 (46.73, 0.15)           2019-04-01 00:00:00+00:00 B              46.73   
                                                                                   S              46.73   
2019-03-31 23:00:46.1

In [4]:
pivoted_new['Executed Volume'].sum()/2

27880.95000000092

In [5]:
pivoted_levels_new

,End Validity Date_B,Executed Price & Volume_B,Delivery Start_B,Executed Volume_B,Execution Price_B,Initial ID_B,Instrument Type_B,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,index_B,lead_time_B,End Validity Date_S,Executed Price & Volume_S,Delivery Start_S,Executed Volume_S,Execution Price_S,Initial ID_S,Instrument Type_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,index_S,lead_time_S
0,2019-03-31 23:00:01.357000+00:00,"(37.23, 0.5)",2019-04-01 00:00:00+00:00,0.5,37.23,10391281385,Quarter Hour,2,N,10391281401,1.03913e+10,39.8,0.65,51413,0 days 00:59:58.643000,2019-03-31 23:00:01.357000+00:00,"(37.23, 0.5)",2019-04-01 00:00:00+00:00,0.5,37.23,10391281397,Quarter Hour,1,N,10391281397,NaN,37.23,0.5,51409,0 days 00:59:58.643000
1,2019-03-31 23:00:02.583000+00:00,"(46.73, 0.15)",2019-04-01 00:00:00+00:00,0.15,46.73,10391281385,Quarter Hour,1,N,10391281419,1.03913e+10,46.73,0.15,51422,0 days 00:59:57.417000,2019-03-31 23:00:02.583000+00:00,"(46.73, 0.15)",2019-04-01 00:00:00+00:00,0.15,46.73,10391281396,Quarter Hour,2,N,10391281396,NaN,46.73,0.5,51408,0 days 00:59:57.417000
2,2019-03-31 23:00:46.106000+00:00,"(43.65, 0.15)",2019-04-01 00:00:00+00:00,0.15,43.65,10391281545,Quarter Hour,1,N,10391281545,NaN,43.65,0.15,51509,0 days 00:59:13.894000,2019-03-31 23:00:46.106000+00:00,"(43.65, 0.15)",2019-04-01 00:00:00+00:00,0.15,43.65,10391281772,Quarter Hour,1,N,10391281772,NaN,43.65,0.15,51649,0 days 00:59:13.894000
3,2019-03-31 23:01:40.633000+00:00,"(26.3, 1.0)",2019-04-01 00:00:00+00:00,1,26.3,10391281822,Hour,2,N,10391281822,NaN,26.3,3,51688,0 days 00:58:19.367000,2019-03-31 23:01:40.633000+00:00,"(26.3, 1.0)",2019-04-01 00:00:00+00:00,1,26.3,10391281997,Hour,1,N,10391281997,NaN,25.3,1,51811,0 days 00:58:19.367000
4,2019-03-31 23:01:40.733000+00:00,"(26.3, 1.0)",2019-04-01 00:00:00+00:00,1,26.3,10391281822,Hour,2,N,10391281822,NaN,26.3,2,51689,0 days 00:58:19.267000,2019-03-31 23:01:40.733000+00:00,"(26.3, 1.0)",2019-04-01 00:00:00+00:00,1,26.3,10391281999,Hour,1,N,10391281999,NaN,25.3,1,51812,0 days 00:58:19.267000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19125,2019-04-01 23:13:00.823000+00:00,"(34.2, 0.1)",2019-04-01 23:45:00+00:00,0.1,34.2,10391875976,Quarter Hour,1,N,10391876818,1.03919e+10,34.2,0.1,614619,0 days 00:31:59.177000,2019-04-01 23:13:00.823000+00:00,"(34.2, 0.1)",2019-04-01 23:45:00+00:00,0.1,34.2,10391876988,Quarter Hour,2,N,10391876988,NaN,33.7,1.575,614653,0 days 00:31:59.177000
19126,2019-04-01 23:13:01.088000+00:00,"(33.6, 1.375)",2019-04-01 23:45:00+00:00,1.375,33.6,10391873113,Quarter Hour,2,N,10391876309,1.03919e+10,33.6,1.575,614532,0 days 00:31:58.912000,2019-04-01 23:13:01.088000+00:00,"(33.6, 1.375)",2019-04-01 23:45:00+00:00,1.375,33.6,10391876988,Quarter Hour,1,N,10391876991,1.03919e+10,33.1,1.375,614656,0 days 00:31:58.912000
19127,2019-04-01 23:14:01.121000+00:00,"(35.0, 1.25)",2019-04-01 23:45:00+00:00,1.25,35,10391877226,Quarter Hour,1,N,10391877226,NaN,35,1.25,614686,0 days 00:30:58.879000,2019-04-01 23:14:01.121000+00:00,"(35.0, 1.25)",2019-04-01 23:45:00+00:00,1.25,35,10391876318,Quarter Hour,2,N,10391876318,NaN,35,1.25,614541,0 days 00:30:58.879000
19128,2019-04-01 23:14:01.839000+00:00,"(35.0, 0.025)",2019-04-01 23:45:00+00:00,0.025,35,10391876998,Quarter Hour,1,N,10391877229,1.03919e+10,35.5,0.025,614688,0 days 00:30:58.161000,2019-04-01 23:14:01.839000+00:00,"(35.0, 0.025)",2019-04-01 23:45:00+00:00,0.025,35,10391876318,Quarter Hour,2,N,10391876318,NaN,35,1.25,614539,0 days 00:30:58.161000


In [6]:
pivoted_levels_new[pivoted_levels_new['Order ID_B']==10391282323]

,End Validity Date_B,Executed Price & Volume_B,Delivery Start_B,Executed Volume_B,Execution Price_B,Initial ID_B,Instrument Type_B,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,index_B,lead_time_B,End Validity Date_S,Executed Price & Volume_S,Delivery Start_S,Executed Volume_S,Execution Price_S,Initial ID_S,Instrument Type_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,index_S,lead_time_S
8,2019-03-31 23:03:28.937000+00:00,"(26.7, 4.9)",2019-04-01 00:00:00+00:00,4.9,26.7,10391282323,Hour,1,N,10391282323,NaN,26.7,4.9,52206,0 days 00:56:31.063000,2019-03-31 23:03:28.937000+00:00,"(26.7, 4.9)",2019-04-01 00:00:00+00:00,4.9,26.7,10391282322,Hour,2,N,10391282322,NaN,26.7,23.1,52202,0 days 00:56:31.063000


In [7]:
pivoted_levels_new[pivoted_levels_new['Initial ID_S']==10391281397]

,End Validity Date_B,Executed Price & Volume_B,Delivery Start_B,Executed Volume_B,Execution Price_B,Initial ID_B,Instrument Type_B,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,index_B,lead_time_B,End Validity Date_S,Executed Price & Volume_S,Delivery Start_S,Executed Volume_S,Execution Price_S,Initial ID_S,Instrument Type_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,index_S,lead_time_S
0,2019-03-31 23:00:01.357000+00:00,"(37.23, 0.5)",2019-04-01 00:00:00+00:00,0.5,37.23,10391281385,Quarter Hour,2,N,10391281401,1.03913e+10,39.8,0.65,51413,0 days 00:59:58.643000,2019-03-31 23:00:01.357000+00:00,"(37.23, 0.5)",2019-04-01 00:00:00+00:00,0.5,37.23,10391281397,Quarter Hour,1,N,10391281397,NaN,37.23,0.5,51409,0 days 00:59:58.643000


In [8]:
pivoted_levels_new[pivoted_levels_new['Instrument Type_B']=='Hour']['Executed Volume_S'].sum()

22601.599999999995

In [9]:
# loading the daily csv file at day
df_new = read_epex_file('../data/external/EPEX_spot_continous_2019/DE Continuous Orders 2019-04/DE Continuous Orders 20190402.csv')

# filtering the daily csv file for the window of time of interest
df_filtered_new = filter_lead_time(df_new)

# derive transactions
pivoted_new, pivoted_levels_new = extract_transactions(df_filtered_new)

In [10]:
pivoted_levels_new[pivoted_levels_new['Instrument Type_B']=='Hour']['Executed Volume_S'].sum()

27128.60000000007

In [16]:
print(pivoted_levels_new.columns.values)

['End Validity Date_B' 'Executed Price & Volume_B' 'Delivery Start_B'
 'Executed Volume_B' 'Execution Price_B' 'Initial ID_B'
 'Instrument Type_B' 'Is Executed_B' 'Is block_B' 'Order ID_B'
 'Parent ID_B' 'Price_B' 'Volume_B' 'index_B' 'lead_time_B'
 'End Validity Date_S' 'Executed Price & Volume_S' 'Delivery Start_S'
 'Executed Volume_S' 'Execution Price_S' 'Initial ID_S'
 'Instrument Type_S' 'Is Executed_S' 'Is block_S' 'Order ID_S'
 'Parent ID_S' 'Price_S' 'Volume_S' 'index_S' 'lead_time_S']


In [17]:
df_filtered_new[(df_filtered_new['Delivery Start']=='2019-04-01 00:00:00+00:00')&
                        (df_filtered_new['Instrument Type']=='Hour')]

,index,Order ID,Initial ID,Parent ID,Start Validity Date,End Validity Date,Cancelling Date,Delivery Date,Instrument Type,Delivery Instrument,Is block,Area,Side,Price,Volume,Is Executed,Execution Price,Executed Volume,Executed Price & Volume,Delivery Start,lead_time
51410,51410,10391281398,10391281398,NaN,2019-03-31 23:00:01.281000+00:00,2019-03-31 23:08:14.735000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-TPS,B,26.00,10.0,1,26.00,10.0,"(26.0, 10.0)",2019-04-01 00:00:00+00:00,0 days 00:51:45.265000
51590,51590,10391281708,10391281708,NaN,2019-03-31 23:18:45.651000+00:00,2019-03-31 23:18:46.792000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-AMP,B,21.10,8.0,2,21.10,8.0,"(21.1, 8.0)",2019-04-01 00:00:00+00:00,0 days 00:41:13.208000
51592,51592,10391281708,10391281708,NaN,2019-03-31 23:18:46.792000+00:00,2019-03-31 23:18:47.628000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-AMP,B,21.10,10.0,2,21.10,10.0,"(21.1, 10.0)",2019-04-01 00:00:00+00:00,0 days 00:41:12.372000
51594,51594,10391281708,10391281708,NaN,2019-03-31 23:18:47.628000+00:00,2019-03-31 23:18:48.273000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-AMP,B,21.10,1.0,1,21.10,1.0,"(21.1, 1.0)",2019-04-01 00:00:00+00:00,0 days 00:41:11.727000
51595,51595,10391281708,10391281708,NaN,2019-03-31 23:00:34.345000+00:00,2019-03-31 23:18:39.772000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-AMP,B,21.10,10.0,2,21.10,1.1,"(21.1, 1.1)",2019-04-01 00:00:00+00:00,0 days 00:41:20.228000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57610,57610,10391288388,10391288388,NaN,2019-03-31 23:29:08.459000+00:00,2019-03-31 23:29:08.558000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-AMP,B,23.10,30.0,2,23.10,5.0,"(23.1, 5.0)",2019-04-01 00:00:00+00:00,0 days 00:30:51.442000
57613,57613,10391288390,10391288390,NaN,2019-03-31 23:29:08.558000+00:00,2019-03-31 23:29:08.558000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-AMP,S,23.00,5.0,1,23.10,5.0,"(23.1, 5.0)",2019-04-01 00:00:00+00:00,0 days 00:30:51.442000
57812,57812,10391288549,10391288175,1.039129e+10,2019-03-31 23:29:50.338000+00:00,2019-03-31 23:29:50.338000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-TPS,B,23.99,0.6,1,23.99,0.6,"(23.99, 0.6)",2019-04-01 00:00:00+00:00,0 days 00:30:09.662000
57814,57814,10391288550,10391288205,1.039129e+10,2019-03-31 23:29:50.421000+00:00,2019-03-31 23:29:59.525000+00:00,NaT,2019-04-01 00:00:00+00:00,Hour,0 days,N,DE-TPS,S,23.99,4.8,2,23.99,4.0,"(23.99, 4.0)",2019-04-01 00:00:00+00:00,0 days 00:30:00.475000


In [18]:
df_filtered_new[(df_filtered_new['Is Executed']!=0)&(df_filtered_new['Side']=='B')]['Executed Volume'].sum()

27880.95

In [19]:
df_filtered_new[(df_filtered_new['Is Executed']!=0)&(df_filtered_new['Side']=='S')]['Executed Volume'].sum()

27880.95

In [20]:
df_filtered_new[(df_filtered_new['Is Executed']!=0)
                &(df_filtered_new['Side']=='S')
               &(df_filtered_new['Instrument Type']=='Hour')]['Executed Volume'].sum()

22601.6

### NTC Investigation

In [17]:
df_updated_transactions=pd.read_csv("../data/processed/EPEX_spot_continous_complete_pipeline_2019_17-04-2021 18:38:10/2019-04/Updated Transactions/DE_20190425.csv",)

In [18]:
df_updated_transactions.head()

,index,End Validity Date,Executed Price & Volume,Delivery Start,Executed Volume,Execution Price,Initial ID_B,Instrument Type,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,index_B,lead_time_B,Initial ID_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,index_S,lead_time_S,weekly_hydro_marginal_price_selling,weekly_hydro_marginal_price_pumping,possible_match_selling,possible_match_pumping,match_binary_outcome_pumping,match_binary_outcome_selling,A posteriori Execution Price
0,0,2019-04-24 23:00:04.214000+00:00,"(28.1, 0.325)",2019-04-25 00:00:00+00:00,0.325,28.10,10405822351,Quarter Hour,1,N,10405822379,1.040582e+10,81.0,0.325,65408,0 days 00:59:55.786000,10405822363,2,N,10405822363,NaN,28.10,6.0,65397,0 days 00:59:57.109000,35.651908,24.956336,False,False,0,0,28.100000
1,3,2019-04-24 23:01:33.033000+00:00,"(24.5, 0.6)",2019-04-25 00:00:00+00:00,0.600,24.50,10405823391,Hour,2,N,10405823391,NaN,24.8,3.000,66328,0 days 00:58:26.967000,10405822460,1,N,10405823175,1.040582e+10,24.50,0.6,66115,0 days 00:58:52.098000,35.651908,24.956336,False,True,1,0,24.956336
2,5,2019-04-24 23:01:33.033000+00:00,"(24.5, 5.0)",2019-04-25 00:00:00+00:00,5.000,24.50,10405823391,Hour,2,N,10405823391,NaN,24.8,8.000,66325,0 days 00:58:26.967000,10405823163,2,N,10405823163,NaN,24.50,5.0,66102,0 days 00:58:52.776000,35.651908,24.956336,False,True,1,0,24.956336
3,4,2019-04-24 23:01:33.033000+00:00,"(24.5, 2.4)",2019-04-25 00:00:00+00:00,2.400,24.50,10405823391,Hour,1,N,10405823391,NaN,24.8,2.400,66329,0 days 00:58:26.967000,10405823163,2,N,10405823163,NaN,24.50,5.0,66103,0 days 00:58:26.967000,35.651908,24.956336,False,True,1,0,24.956336
4,2,2019-04-24 23:01:33.033000+00:00,"(24.49, 5.0)",2019-04-25 00:00:00+00:00,5.000,24.49,10405823391,Hour,2,N,10405823391,NaN,24.8,13.000,66326,0 days 00:58:26.967000,10405823225,1,N,10405823225,NaN,24.49,5.0,66157,0 days 00:58:49.601000,35.651908,24.956336,False,True,1,0,24.956336


In [19]:
df_updated_transactions['match_binary_outcome_selling'].value_counts()

1    22903
0     7561
2     1164
Name: match_binary_outcome_selling, dtype: int64

In [42]:
df_updated_transactions[df_updated_transactions['match_binary_outcome_selling']==2]

,index,End Validity Date,Executed Price & Volume,Delivery Start,Executed Volume,Execution Price,Initial ID_B,Instrument Type,Is Executed_B,Is block_B,Order ID_B,Parent ID_B,Price_B,Volume_B,index_B,lead_time_B,Initial ID_S,Is Executed_S,Is block_S,Order ID_S,Parent ID_S,Price_S,Volume_S,index_S,lead_time_S,weekly_hydro_marginal_price_selling,weekly_hydro_marginal_price_pumping,possible_match_selling,possible_match_pumping,match_binary_outcome_pumping,match_binary_outcome_selling,A posteriori Execution Price
3555,3557,2019-04-25 01:14:00.691000+00:00,"(37.99, 0.125)",2019-04-25 01:45:00+00:00,0.125,37.99,10405888508,Quarter Hour,1,N,10405890981,1.040589e+10,38.1,0.125,136169,0 days 00:30:59.309000,10405879775,2,N,10405885344,1.040588e+10,37.99,0.425,130210,0 days 00:30:59.309000,35.651908,24.956336,True,False,0,2,37.99
3556,3558,2019-04-25 01:14:00.691000+00:00,"(37.99, 0.55)",2019-04-25 01:45:00+00:00,0.550,37.99,10405888981,Quarter Hour,1,N,10405890979,1.040589e+10,38.1,0.550,136167,0 days 00:30:59.309000,10405879775,2,N,10405885344,1.040588e+10,37.99,1.000,130208,0 days 00:39:18.599000,35.651908,24.956336,True,False,0,2,37.99
3557,3556,2019-04-25 01:14:00.691000+00:00,"(37.99, 0.025)",2019-04-25 01:45:00+00:00,0.025,37.99,10405888490,Quarter Hour,1,N,10405890980,1.040589e+10,38.1,0.025,136168,0 days 00:30:59.309000,10405879775,2,N,10405885344,1.040588e+10,37.99,0.450,130211,0 days 00:30:59.309000,35.651908,24.956336,True,False,0,2,37.99
3558,3555,2019-04-25 01:14:00.691000+00:00,"(37.6, 0.125)",2019-04-25 01:45:00+00:00,0.125,37.60,10405888981,Quarter Hour,2,N,10405890979,1.040589e+10,38.1,0.675,136166,0 days 00:30:59.309000,10405874622,1,N,10405874622,NaN,37.60,0.125,119251,0 days 00:59:32.832000,35.651908,24.956336,True,False,0,2,37.60
4124,4126,2019-04-25 01:26:15.946000+00:00,"(36.1, 0.25)",2019-04-25 02:15:00+00:00,0.250,36.10,10405897861,Quarter Hour,1,N,10405897861,NaN,36.1,0.250,143704,0 days 00:48:44.054000,10405897096,2,N,10405897096,NaN,36.10,0.375,142695,0 days 00:49:24.497000,35.651908,24.956336,True,False,0,2,36.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7275,7275,2019-04-25 03:14:00.910000+00:00,"(53.3, 0.1)",2019-04-25 03:45:00+00:00,0.100,53.30,10405965484,Quarter Hour,2,N,10405965484,NaN,53.3,0.350,218016,0 days 00:31:28.290000,10405965687,1,N,10405965687,NaN,53.20,0.100,218237,0 days 00:30:59.090000,35.651908,24.956336,True,False,0,2,53.30
7284,7284,2019-04-25 03:14:28.317000+00:00,"(53.2, 0.25)",2019-04-25 03:45:00+00:00,0.250,53.20,10405960756,Quarter Hour,2,N,10405965370,1.040596e+10,53.2,0.725,217893,0 days 00:31:43.834000,10405960262,1,N,10405966005,1.040597e+10,53.20,0.250,218587,0 days 00:30:31.683000,35.651908,24.956336,True,False,0,2,53.20
7285,7285,2019-04-25 03:14:28.317000+00:00,"(53.3, 0.25)",2019-04-25 03:45:00+00:00,0.250,53.30,10405965484,Quarter Hour,1,N,10405965484,NaN,53.3,0.250,218017,0 days 00:30:59.090000,10405960262,2,N,10405966005,1.040597e+10,53.20,0.500,218586,0 days 00:30:31.683000,35.651908,24.956336,True,False,0,2,53.30
7286,7286,2019-04-25 03:14:28.708000+00:00,"(53.2, 0.475)",2019-04-25 03:45:00+00:00,0.475,53.20,10405960756,Quarter Hour,1,N,10405965370,1.040596e+10,53.2,0.475,217892,0 days 00:30:31.683000,10405966012,2,N,10405966012,NaN,53.20,1.250,218594,0 days 00:30:31.292000,35.651908,24.956336,True,False,0,2,53.20


In [31]:
mask = df_updated_transactions['match_binary_outcome_selling'] == 1
df_updated_transactions_f = df_updated_transactions[mask]
times = pd.to_datetime(
    df_updated_transactions_f['Delivery Start'])

gb = df_updated_transactions_f['Executed Volume'].groupby(
    times.dt.hour)

In [32]:
gb.groups.keys()

dict_keys([2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23])

In [33]:
gb.sum()

Delivery Start
2     1950.700
3     4224.725
4     5195.575
5      669.675
6     2332.550
7     1032.900
8     2653.050
9     3582.850
10    2704.825
11    3470.075
12    2026.425
13    1715.375
14    2624.375
15     896.625
16    1244.075
17    1605.375
18    1613.475
19    2490.750
20    3464.250
21    2755.700
22     257.775
23    1170.100
Name: Executed Volume, dtype: float64

In [36]:
NTC = read_NTC_file("../data/external/NTC_DEandCH_2019.csv")
NTC = NTC_preparation(NTC)

/Users/gianlucamancini/Documents/eth_codes/case_study/case_study_2_alpiq/env/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [39]:
NTC_selection = NTC[(NTC['start_time'].dt.day == 25) &  (NTC['start_time'].dt.month == 4)]

In [41]:
NTC_selection.head(15)

,Date from,Time from,Time to,CH to DE_Utilized capacity (MW),CH to DE_Actual value (MW),De to CH_Utilized capacity (MW),DE to CH_Actual value (MW),Intraday has taken place,Time from hour,Time from min,start_time,Time to hour,Time to min,end_time,CH to DE_Actual value (MW) update,DE to CH_Actual value (MW) update
10940,2019-04-25 00:00:00+00:00,00:00,00:15,4385,0,415,385,Yes,0,0,2019-04-25 00:00:00+00:00,0,15,2019-04-25 00:15:00+00:00,0,385
10941,2019-04-25 00:00:00+00:00,00:15,00:30,4385,0,415,385,Yes,0,15,2019-04-25 00:15:00+00:00,0,30,2019-04-25 00:30:00+00:00,0,385
10942,2019-04-25 00:00:00+00:00,00:30,00:45,4385,0,415,385,Yes,0,30,2019-04-25 00:30:00+00:00,0,45,2019-04-25 00:45:00+00:00,0,385
10943,2019-04-25 00:00:00+00:00,00:45,01:00,4385,0,415,385,Yes,0,45,2019-04-25 00:45:00+00:00,1,0,2019-04-25 01:00:00+00:00,0,385
10944,2019-04-25 00:00:00+00:00,01:00,01:15,4481,0,318,482,Yes,1,0,2019-04-25 01:00:00+00:00,1,15,2019-04-25 01:15:00+00:00,0,482
10945,2019-04-25 00:00:00+00:00,01:15,01:30,4481,0,318,482,Yes,1,15,2019-04-25 01:15:00+00:00,1,30,2019-04-25 01:30:00+00:00,0,482
10946,2019-04-25 00:00:00+00:00,01:30,01:45,4481,0,318,482,Yes,1,30,2019-04-25 01:30:00+00:00,1,45,2019-04-25 01:45:00+00:00,0,482
10947,2019-04-25 00:00:00+00:00,01:45,02:00,4481,0,318,482,Yes,1,45,2019-04-25 01:45:00+00:00,2,0,2019-04-25 02:00:00+00:00,0,482
10948,2019-04-25 00:00:00+00:00,02:00,02:15,4517,0,253,547,Yes,2,0,2019-04-25 02:00:00+00:00,2,15,2019-04-25 02:15:00+00:00,0,547
10949,2019-04-25 00:00:00+00:00,02:15,02:30,4517,0,253,547,Yes,2,15,2019-04-25 02:15:00+00:00,2,30,2019-04-25 02:30:00+00:00,0,547


In [ ]:
ntc_gb = NTC_selection['CH to DE_Actual value (MW)'].groupby(
            NTC_selection['start_time'].dt.hour)

In [44]:
t=[True, True, False, False]

In [45]:
np.product(t)

0

In [48]:
np.array(t).astype('float')

array([1., 1., 0., 0.])

In [50]:
a=0

if a:
    print('ok')